## Predpriprava

In [33]:
#use "topfind" ;;
#require "jupyter.notebook" ;;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/Users/matija/.opam/default/lib/base64: added to search path
/Users/matija/.opam/default/lib/base64/base64.cma: loaded
/Users/matija/.opam/default/lib/ocaml/compiler-libs: added to search path
/Users/matija/.opam/default/lib/ocaml/compiler-libs/ocamlcommon.cma: loaded
/Users/matija/.opam/default/lib/seq: added to search path
/Users/matija/.opam/default/lib/yojson: added to search path
/Users/matija/.opam/default/lib/yojson/yojson.cma: loaded
/Users/matija/.opam/default/lib/ppx_yojson_conv_lib: added to search path
/Users/matija/.opam/default/lib/ppx_yojson_conv_lib/ppx_yojson_conv_lib.cma: loaded
/Users/matija/.opam/default/lib/ocaml/unix.cma: loaded
/Users/matija/.opam/default/lib/bytes: added to search path
/Users/matija/.opam/default/lib/uuidm: added to search path
/Users/matija/.opam/default/lib/uuidm/uuidm.cma: loaded
/Users/matija/.opam/default/lib/jupyter: added to search path
/Users/matija/.opam/default/lib/jupyter/jupyter.cma: loaded
/Users/matija/.opam/default/lib/result: add

## Zanki `while` in `for`

In [6]:
let vsota = ref 0 in
for i = 1 to 10 do
  vsota := !vsota + i
done;
!vsota

- : int = 55


In [8]:
let i = ref 1 in
let vsota = ref 0 in
while !i <= 10 do
  vsota := !vsota + !i;
  incr i
done;
!vsota

- : int = 55


In [9]:
incr

- : int ref -> unit = <fun>


## Spreminjanje tabel na mestu

In [10]:
let zamenjaj tabela i j =
  let x = tabela.(i) in
  tabela.(i) <- tabela.(j);
  tabela.(j) <- x

val zamenjaj : 'a array -> int -> int -> unit = <fun>


In [11]:
let t = [| 1; 2; 3; 4; 5 |]

val t : int array = [|1; 2; 3; 4; 5|]


In [13]:
t.(3) <- 40

- : unit = ()


In [15]:
t.(40) <- 3

error: runtime_error

In [14]:
t

- : int array = [|1; 2; 3; 40; 5|]


In [16]:
zamenjaj t 0 4

- : unit = ()


In [17]:
t

- : int array = [|5; 2; 3; 40; 1|]


In [18]:
let obrni_na_mestu tabela =
  let n = Array.length tabela in
  for i = 0 to n / 2 do
    zamenjaj tabela i (n - 1 - i)
  done

val obrni_na_mestu : 'a array -> unit = <fun>


In [21]:
let t = [| 10; 20; 30 |] in
obrni_na_mestu t;
t

- : int array = [|30; 20; 10|]


In [23]:
let naredi_obrnjeno tabela =
  let n = Array.length tabela in
  Array.init n (fun i -> tabela.(n - 1 - i))

val naredi_obrnjeno : 'a array -> 'a array = <fun>


## Mešanje

In [67]:
let nakljucno_celo a b =
  a + Random.int (b + 1 - a)

let premesaj tabela =
  let n = Array.length tabela in
  for i = 0 to n - 2 do
    let j = nakljucno_celo (i + 1) (n - 1) in
    zamenjaj tabela i j
  done

val nakljucno_celo : int -> int -> int = <fun>


val premesaj : 'a array -> unit = <fun>


In [31]:
let tabela = Array.init 10 (fun i -> i) in
premesaj tabela;
tabela

- : int array = [|9; 6; 5; 8; 2; 7; 4; 1; 0; 3|]


### Porazdelitev verjetnosti dogodkov

In [34]:
let prikazi_verjetnosti verjetnosti =
  Jupyter_notebook.printf "<table><tr><th>dogodek</th><th>verjetnost</th><th style=\"width: 300px\"></th></tr>";
  verjetnosti
  |> List.iter (fun (x, p) ->
    let percent = (100. *. p) in
    Jupyter_notebook.printf "<tr><td>%s</td><td>%f%%</td><td><div style=\"width: %f%%; height: 1em; background:black\"></div></td></tr>" x percent percent) ;
  Jupyter_notebook.printf "</table>" ;
  Jupyter_notebook.display_formatter "text/html"

val prikazi_verjetnosti :
  (string * float) list -> Jupyter_notebook.display_id = <fun>


In [35]:
prikazi_verjetnosti [("sonce", 0.5); ("dež", 0.1); ("oblačno", 0.2); ("sneg", 0.1)]

dogodek,verjetnost,
sonce,50.000000%,
dež,10.000000%,
oblačno,20.000000%,
sneg,10.000000%,


- : Jupyter_notebook.display_id = <abstr>


In [36]:
let verjetnost_rezultatov poskus stevilo_poskusov =
  let ponovitve = Hashtbl.create 256 in
  for _ = 1 to stevilo_poskusov do
    let rezultat = poskus () in
    if Hashtbl.mem ponovitve rezultat then
      Hashtbl.replace ponovitve rezultat (Hashtbl.find ponovitve rezultat + 1)
    else Hashtbl.add ponovitve rezultat 1
  done;
  let verjetnosti =
    Hashtbl.fold
      (fun rezultat stevilo seznam ->
        (rezultat, float_of_int stevilo /. float_of_int stevilo_poskusov)
        :: seznam)
      ponovitve []
  in
  List.sort (fun (x1, p1) (x2, p2) -> compare x1 x2) verjetnosti

val verjetnost_rezultatov : (unit -> 'a) -> int -> ('a * float) list = <fun>


In [53]:
verjetnost_rezultatov (fun () -> nakljucno_celo 1 6) 100000
|> List.map (fun (met, p) -> (string_of_int met, p))
|> prikazi_verjetnosti

dogodek,verjetnost,
1,16.747000%,
2,16.484000%,
3,16.681000%,
4,16.682000%,
5,16.634000%,
6,16.772000%,


- : Jupyter_notebook.display_id = <abstr>


### Pravičnost mešanja

In [58]:
let verjetnost_permutacij premesaj zacetna_tabela stevilo_poskusov =
  let poskus () =
    let kopija = Array.copy zacetna_tabela in
    let () = premesaj kopija in
    kopija
    |> Array.to_list
    |> String.concat ""
  in
  verjetnost_rezultatov poskus stevilo_poskusov
  |> prikazi_verjetnosti

val verjetnost_permutacij :
  (string array -> unit) ->
  string array -> int -> Jupyter_notebook.display_id = <fun>


In [85]:
let ne_naredi_nic _tabela =
  ()

in

(* dobimo samo prvotno permutacijo *)
verjetnost_permutacij ne_naredi_nic [|"A"; "B"; "C"; "D"|] 10000

dogodek,verjetnost,
ABCD,100.000000%,


- : Jupyter_notebook.display_id = <abstr>


In [90]:
let zamenjuj_pare tabela =
  let n = Array.length tabela in
  for k = 1 to n do
    let i = nakljucno_celo 0 (n - 1)
    and j = nakljucno_celo 0 (n - 1) in
    zamenjaj tabela i j
  done
in

(* dobimo samo prvotno permutacijo *)
verjetnost_permutacij zamenjuj_pare [|"A"; "B"; "C"|] 10000

dogodek,verjetnost,
ABC,18.750000%,
ACB,17.260000%,
BAC,16.880000%,
BCA,15.310000%,
CAB,14.520000%,
CBA,17.280000%,


- : Jupyter_notebook.display_id = <abstr>


### Fisher-Yatesov algoritem

In [74]:
let brez_fiksnih_tock tabela =
  let n = Array.length tabela in
  for i = 0 to n - 2 do
    (* i nikoli ne zamenjamo z i *)
    let j = nakljucno_celo (i + 1) (n - 1) in
    zamenjaj tabela i j
  done

in

verjetnost_permutacij brez_fiksnih_tock [|"A"; "B"; "C"; "D"|] 10000

dogodek,verjetnost,
BCDA,16.820000%,
BDAC,17.240000%,
CADB,16.300000%,
CDBA,16.200000%,
DABC,16.540000%,
DCAB,16.900000%,


- : Jupyter_notebook.display_id = <abstr>


In [91]:
let fisher_yates tabela =
  let n = Array.length tabela in
  for i = 0 to n - 2 do
    let j = nakljucno_celo i (n - 1) in
    zamenjaj tabela i j
  done

in

verjetnost_permutacij fisher_yates [|"A"; "B"; "C"; "D"|] 100000

dogodek,verjetnost,
ABCD,4.071000%,
ABDC,4.204000%,
ACBD,4.156000%,
ACDB,4.239000%,
ADBC,4.215000%,
ADCB,4.226000%,
BACD,4.131000%,
BADC,4.201000%,
BCAD,4.164000%,
BCDA,4.206000%,


- : Jupyter_notebook.display_id = <abstr>


In [93]:
let malo_pokvarjen_fisher_yates tabela =
  let n = Array.length tabela in
  for i = 0 to n - 2 do
    (* izbiramo tudi med že izbranimi elementi *)
    let j = nakljucno_celo 0 (n - 1) in
    zamenjaj tabela i j
  done

in

verjetnost_permutacij malo_pokvarjen_fisher_yates [|"A"; "B"; "C"; "D"|] 100000

dogodek,verjetnost,
ABCD,6.325000%,
ABDC,3.168000%,
ACBD,7.905000%,
ACDB,3.187000%,
ADBC,1.622000%,
ADCB,3.069000%,
BACD,7.771000%,
BADC,3.014000%,
BCAD,7.858000%,
BCDA,7.726000%,


- : Jupyter_notebook.display_id = <abstr>


## Naivni algoritmi za urejanje

## Urejanje z zlivanjem

![](../../zapiski/slike/urejanje-z-zlivanjem.png)

## Hitro urejanje

![](../../zapiski/slike/hitro-urejanje.png)